In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import pandas as pd
import dask
from multiprocess import Pool

In [2]:
def dewpoint_from_q(q, p):
    '''
    q - specific humidity in kg/kg
    p - pressure in hPa
    return: 
    t_d - dewpoint in K
    '''
    M_w = 18.015268 # g/mol
    M_d = 28.96546  # g/mol
    epsilon = M_w /  M_d
    
    ### mixing ratio w 
    w = q / (1-q)
    ### vapour pressure e
    e = p * w / (epsilon + w)
    ### intermediate variable val
    val = np.log(e/6.112)
    ### dewpoint t_d
    t_d = 273.15 + 243.5*val / (17.67 - val)
    return t_d

In [3]:
ds_hus = xr.open_dataset("/scratch/nf33/hk25_LSP/Germany/Darwin/hus_6hourly.nc")


In [4]:
td_arr = np.zeros_like(ds_hus["hus"].values)

In [5]:
for i, plvl in enumerate(ds_hus["pressure"]):
    td_arr[:,i,:,:] = dewpoint_from_q(ds_hus["hus"].sel(pressure=plvl).values,plvl.values/100)

In [6]:
ds_td = ds_hus.copy(deep=True)

In [7]:
ds_td["hus"].values = td_arr
ds_td = ds_td.rename({"hus": "tdew"})

In [8]:
ds_td.to_netcdf("/scratch/nf33/hk25_LSP/Germany/Darwin/tdew_6hourly.nc")